Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Vertex AI Search - REST API 호출로 검색

Feedback : shins777@gmail.com

이 Colab 은 대부분의 고객의 요구사항이면서, 많은 유즈케이스에서의 근간이 되는 지식검색에 대한 예제입니다.
Gronding service 로 Vertex AI Search와 Langchain 을 연동해서 정보를 검색하고 해당 검색된 정보를 Gemini 를 통해서 추론해서 답해주는 형태의 예제입니다.

* Vertex AI Search Manual references
    * https://cloud.google.com/enterprise-search?hl=ko
    * https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search


이 소스에서 지식 검색의 대상은 "정보통신 진흥 및 융합 활성화 등에 관한 특별법" 이며 해당 특별법은 아래 사이트에서 다운받았습니다.
https://www.law.go.kr/

#라이브러리 설치
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [1]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai google-cloud-discoveryengine

Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [3]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.
* 아래 SEARCH_URL Vertex AI Search를 구성 후 Integration 메뉴항목에서 추출된 값입니다. 

In [4]:
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL = "gemini-1.5-pro-preview-0409"
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/servingConfigs/default_search:search"

In [5]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

vertexai.init(project=PROJECT_ID, location=REGION)

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()


### Vertex AI Search 검색

아래 코드는 REST API를 통해서 Verext AI Search에 직접 검색 쿼리를 보내는 예제입니다.
검색에 활용되는 패러미터는 ContentSearchSpec 에 따릅니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec

특히 ExtractiveContentSpec은 검색 컨텐츠를 좀더 상세하게 얻을수 있는 설정입니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec.ExtractiveContentSpec

In [6]:
def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{

            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }              
      },
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text


#### Search 결과 파싱 로직
아래 로직은 검색된 결과를 파싱하는 로직입니다.

In [7]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']

            #print(chunk)
            # print(f"content {chunk['content']}")

            # print(f"documentMetadata {chunk['documentMetadata']}")
            # print(f"documentMetadata {chunk['documentMetadata']['title']}")
            # print(f"documentMetadata {chunk['documentMetadata']['uri']}")

            # print(f"documentMetadata {chunk['pageSpan']['pageStart']}")
            # print(f"documentMetadata {chunk['pageSpan']['pageEnd']}")

            # print(f"previousChunks {chunk['chunkMetadata']['previousChunks'][0]['content']}")
            # print(f"nextChunks {chunk['chunkMetadata']['nextChunks'][0]['content']}")

            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['pageSpan'] = f"{chunk['pageSpan']['pageStart']} ~ {chunk['pageSpan']['pageEnd']}"
            item['content'] = chunk['content']

            # chunk 는 현재 Contents에 가까운것 부터 나타남.
            p_chunks = chunk['chunkMetadata']['previousChunks']
            for p_chunk in p_chunks:
                item['content'] = p_chunk['content'] +"\n"+ item['content']

            n_chunks = chunk['chunkMetadata']['nextChunks']
            for n_chunk in n_chunks:
                item['content'] = item['content'] +"\n"+ n_chunk['content']

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

In [8]:
question = "개인정보 보호법에 대해서 설명해주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)


{
  "results": [
    {
      "chunk": {
        "name": "projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/branches/0/documents/1/chunks/c3",
        "id": "c3",
        "content": "\n# 개인정보 보호법\n3. 개인정보의 처리 여부를 확인하고 개인정보에 대한 열람(사본의 발급을 포함한다. 이하 같다) 및 전송을 요구할 권리 4. 개인정보의 처리 정지, 정정·삭제 및 파기를 요구할 권리 5. 개인정보의 처리로 인하여 발생한 피해를 신속하고 공정한 절차에 따라 구제받을 권리 6. 완전히 자동화된 개인정보 처리에 따른 결정을 거부하거나 그에 대한 설명 등을 요구할 권리 제5조(국가 등의 책무) ① 국가와 지방자치단체는 개인정보의 목적 외 수집, 오용·남용 및 무분별한 감시·추적 등에 따른 폐해를 방지하여 인간의 존엄과 개인의 사생활 보호를 도모하기 위한 시책을 강구하여야 한다. ② 국가와 지방자치단체는 제4조에 따른 정보주체의 권리를 보호하기 위하여 법령의 개선 등 필요한 시책을 마련하 여야 한다. ③ 국가와 지방자치단체는 만 14세 미만 아동이 개인정보 처리가 미치는 영향과 정보주체의 권리 등을 명확하게 알 수 있도록 만 14세 미만 아동의 개인정보 보호에 필요한 시책을 마련하여야 한다. \u003c신설 2023. 3. 14.\u003e ④ 국가와 지방자치단체는 개인정보의 처리에 관한 불합리한 사회적 관행을 개선하기 위하여 개인정보처리자의 자 율적인 개인정보 보호활동을 존중하고 촉진·지원하여야 한다. \u003c개정 2023. 3. 14.\u003e ⑤ 국가와 지방자치단체는 개인정보의 처리에 관한 법령 또는 조례를 적용할 때에는 정보주체의 권리가 보장될 수 있도록 개인정보 보호 원칙에 맞게 적용하여야 한다. \u003c개정

In [9]:
context = parse_chunks(searched_ctx)

print(context)

{'results-0': {'title': '개인정보 보호법(법률)(제19234호)(20240315)', 'uri': 'gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'pageSpan': '1 ~ 2', 'content': '\n# 제1장 총칙\n개인정보 보호법 \n\n# 개인정보 보호법\n[시행 2024. 3. 15.] [법률 제19234호, 2023. 3. 14., 일부개정] 개인정보보호위원회 (개인정보보호정책과 - 법령 제개정) 02-2100-3057 개인정보보호위원회 (심사총괄담당관 - 법령 해석) 02-2100-3043 \n_START_OF_TABLE_\nTABLE_IN_MARKDOWN:\n| 1) O OO 12 947% 2 MS MOOTOD WZW 461, 407 7 º 7 700% 24OZ C†. <7☎2014. 3. 24.> 2) ± 809 €24% %. <7&2014. 3. 24., 2020. 2. 4., 2023. 3. 14.> 1. Z 43 449% ± 93 9ct. 7. SH, F05#H$ # gỗ E%E80 7HOlg 20|E → ME SE L. đHE JEEOFE E5 7|9lg g0|E + SHEEIE CHẾ JHQ Ê7 3#öO 20|E + XE JE. 0| 3& 27 22ª + XE QYE 42 149 27 718 57% ± C ^DEE 12, 48,7± 5 ON 120). (†. 79 ± 49 ||1º|20|| 272TON 20 VEZ ±017 º ±7 ¿40 18.12 A ↑ "7+ggy″2 µ†) 1º|2. “7|2|2 70 271 41874 OY S± Q¥% ± 5% WHOZ *7 %%+94 #EDE X% %. 2. ”2″ ZEO TZ, H, α7, ª², 73, Nº, Yº, 72, 22, 24, ±ª, &&(IE), ±7, ⁰8, 1², 27, 17|(1), 1 40||| 02 Oct. 3. "|" 

### Gemini Pro 실행 - Vertex AI Search as a Grounding Service

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [10]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=question)

print(f"Prompt : {prompt}")

response = gemini_pro.invoke(prompt)
display(Markdown(response))

Prompt : 

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {'results-0': {'title': '개인정보 보호법(법률)(제19234호)(20240315)', 'uri': 'gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'pageSpan': '1 ~ 2', 'content': '\n# 제1장 총칙\n개인정보 보호법 \n\n# 개인정보 보호법\n[시행 2024. 3. 15.] [법률 제19234호, 2023. 3. 14., 일부개정] 개인정보보호위원회 (개인정보보호정책과 - 법령 제개정) 02-2100-3057 개인정보보호위원회 (심사총괄담당관 - 법령 해석) 02-2100-3043 \n_START_OF_TABLE_\nTABLE_IN_MARKDOWN:\n| 1) O OO 12 947% 2 MS MOOTOD WZW 461, 407 7 º 7 700% 24OZ C†. <7☎2014. 3. 24.> 2) ± 809 €24% %. <7&2014. 3. 24., 2020. 2. 4., 2023. 3. 14.> 1. Z 43 449% ± 93 9ct. 7. SH, F05#H$ # gỗ E%E80 7HOlg 20|E → ME SE L. đHE JEEOFE E5 7|9lg g0|E + SHEEIE CHẾ JHQ Ê7 3#öO 20|E + XE JE. 0| 3& 27 22ª + XE QYE 42 149 27 718 57% ± C ^DEE 12, 48,7± 5 ON 120). (†. 79 ± 49 ||1º|20|| 272TON 20 VEZ ±017 º ±7 ¿40 18.12 A ↑ "7+ggy″2 µ†) 1º|2. “7|2|2 70 271 41874 OY S± Q¥% ± 5% WHOZ *

## 개인정보 보호법에 대한 설명

2024년 3월 15일 시행된 개인정보 보호법(법률 제19234호)은 개인정보의 목적 외 수집, 오용, 남용 및 무분별한 감시 추적 등에 따른 폐해를 방지하여 인간의 존엄과 개인의 사생활 보호를 도모하는 것을 목적으로 합니다. 

**주요 내용은 다음과 같습니다.**

1. **개인정보 처리자의 의무**: 개인정보처리자는 개인정보 처리 목적에 필요한 범위에서 정확하고 완전하며 최신의 정보를 보장해야 합니다. 또한, 정보주체의 권리 침해 가능성과 위험 정도를 고려하여 안전하게 정보를 관리하고, 처리 방침 등을 공개해야 합니다.
2. **정보주체의 권리**: 정보주체는 자신의 개인정보 처리와 관련하여 정보 제공 요구, 동의 여부 결정, 처리 여부 확인, 열람 및 전송 요구, 처리 정지, 정정, 삭제 및 파기 요구, 피해 구제, 자동화된 처리 결정 거부 및 설명 요구 등의 권리를 가집니다.
3. **국가 등의 책무**: 국가와 지방자치단체는 개인정보 보호를 위한 시책을 강구하고, 정보주체의 권리 보호를 위한 법령 개선 등 필요한 시책을 마련해야 합니다. 또한, 만 14세 미만 아동의 개인정보 보호를 위한 시책 마련, 개인정보처리자의 자율적인 개인정보 보호활동 존중 및 촉진, 개인정보 보호 원칙에 맞는 법령 적용 등의 책무를 가지고 있습니다.
4. **다른 법률과의 관계**: 개인정보 처리 및 보호에 관하여 다른 법률에 특별한 규정이 있는 경우를 제외하고는 개인정보 보호법이 우선적으로 적용됩니다. 다른 법률을 제정하거나 개정할 때에도 개인정보 보호법의 목적과 원칙에 맞도록 해야 합니다.

**이 외에도 개인정보 보호위원회의 설립 및 운영, 개인정보 보호 정책 수립, 개인정보 침해 요인 평가, 개인정보 처리 방침 수립 및 공개, 개인정보 보호책임자 지정, 개인정보 파일 등록 및 공개, 개인정보 영향평가 등에 대한 내용을 담고 있습니다.** 

자세한 내용은 법제처 국가법령정보센터에서 "개인정보 보호법"을 검색하여 확인하실 수 있습니다. 
